In [20]:
from datetime import date, datetime, timezone, timedelta
from urllib.request import Request, urlopen
import requests
from bs4 import BeautifulSoup
import os
import re
import time
import random
import pytz
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

In [21]:
driver = webdriver.Firefox(executable_path='C:/Users/juliakimjk/Downloads/geckodriver.exe')

# Список ссылок по дням

In [3]:
BEGIN_RIA = 730774
DELTA_RIA = int(re.search('[0-9]+', str(date.today() - date.fromordinal(BEGIN_RIA))).group(0))

In [4]:
def overall_links_ria():
    datel = []
    for i in range(BEGIN_RIA, BEGIN_RIA+DELTA_RIA):
        d = date.fromordinal(i)
        dd = d.strftime("https://ria.ru/20%y%m%d/")
        datel.append(dd)
    return datel

In [60]:
6286+366

6652

In [52]:
datel = overall_links_ria()

In [7]:
datel = overall_links_ria()

In [6]:
len(datel)

6797

In [61]:
datel[6652]

'https://ria.ru/20200102/'

# Формирование ссылок на статьи

In [22]:
def ria_year_links(links_list, year, begin, end):
    for i in links_list[begin:end]:
        driver.get(i)
        try:
            driver.find_element_by_xpath('//div[@class="list-more"]').click()
        except NoSuchElementException as ex:
            pass
        
        # Get scroll height
        last_height = driver.execute_script("return document.documentElement.scrollHeight")
        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0,document.documentElement.scrollHeight);")

            # Wait to load page
            time.sleep(1.5)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.documentElement.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        results = driver.find_elements_by_xpath('//a[@class="list-item__title color-font-hover-only"]')
        ps = []
        for r in results:
            ps.append(r.get_attribute('href'))
        with open('ria{}.txt'.format(year), 'a', encoding='UTF-8') as f:
                f.write('\n' + '\n'.join(ps))

In [62]:
ria_year_links(datel, 2020 6286, 6651) #за 2020 надо пересчитать и запустить

# Для скачивания txt
1 - Дата
2 - Название
3 - Аннотация
4 - Текст статьи

In [35]:
def ria_downloader(links_file, corpus_dir, begin, end):
    with open(links_file, 'r', encoding='UTF-8') as f:
        links = f.readlines()[begin:end]
    if os.path.exists(corpus_dir) and os.path.isdir(corpus_dir):
        os.chdir(corpus_dir)
    else:
        os.mkdir(corpus_dir)
        os.chdir(corpus_dir)
    for link in links:
        if link.startswith('https://ria.ru/'):
            res = requests.get(link, {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})
            link0 = link.strip('\n')
            link1 = re.sub(r'/', '-', link0[15:-4])
            if os.path.exists('ria' + link1 + 'txt'):
                pass
            else:
                if res.status_code == 200:
                    try:
                        req = requests.get(link)
                        soup = BeautifulSoup(req.content, 'lxml')
                        title = soup.find('h1', {'class': 'article__title'}).text
                        time = soup.find('div', {'class': 'article__info-date'}).text
                        year = int(time[12:17])
                        month = int(time[9:11])
                        day = int(time[6:8])
                        hour = int(time[0:2])
                        minute = int(time[3:6])
                        dt = datetime(year, month, day, hour, minute, tzinfo=timezone(timedelta(hours=3))).isoformat()
                        annotation = soup.find('div', {'class': 'article__announce-text'}).text
                        txt = soup.find_all('div', {'class': 'article__text'})
                        txts = []
                        for t in txt:
                            txts.append(t.text)
                        with open('ria' + link1 + 'txt', 'w', encoding="UTF-8") as f:
                            f.write(dt + '\n' + title + '\n' + annotation + '\n' + ' '.join(txts))
                        randsleep = random.randint(3,9)/10
                        time.sleep (randsleep)
                    except (TypeError, AttributeError) as e:
                            pass

# На случай обрыва соединения

In [96]:
for i in range(0, 24000):
    if datel[i] == 'https://ria.ru/20031104/':
        print(i)

749


19148 ria2004 - остановилась тут

In [ ]:
%%time
ria_downloader('C:/Users/juliakimjk/Desktop/kursovaya/ria2011.txt', 'D:/corp/RIA/RIA/ria2011', 0, 40000)

In [26]:
os.chdir(r'C:/Users/juliakimjk/Desktop/kursovaya/')

In [32]:
link = 'https://ria.ru/20110101/316355819.html'
res = requests.get(link, {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})
link1 = re.sub(r'/', '-', link[15:-4])
if os.path.exists('ria' + link1 + 'txt'):
    pass
else:
    if res.status_code == 200:
        try:
            req = requests.get(link)
            soup = BeautifulSoup(req.content, 'lxml')
            title = soup.find('h1', {'class': 'article__title'}).text
            time = soup.find('div', {'class': 'article__info-date'}).text
            year = int(time[12:17])
            month = int(time[9:11])
            day = int(time[6:8])
            hour = int(time[0:2])
            minute = int(time[3:6])
            dt = datetime(year, month, day, hour, minute, tzinfo=timezone(timedelta(hours=3))).isoformat()
            annotation = soup.find('div', {'class': 'article__announce-text'}).text
            txt = soup.find_all('div', {'class': 'article__text'})
            txts = []
            for t in txt:
                txts.append(t.text)
            with open('ria' + link1 + 'txt', 'w', encoding="UTF-8") as f:
                f.write(dt + '\n' + title + '\n' + annotation + '\n' + ' '.join(txts))
            randsleep = random.randint(3,9)/10
            time.sleep (randsleep)
        except (TypeError, AttributeError) as e:
            pass

In [31]:
os.getcwd()

'D:\\corp\\RIA\\RIA\\ria2011'